In [1]:
import dask.dataframe as dd
import pandas as pd


In [2]:

# Carregar a tabela com d_x colunas
sales = dd.read_csv('../data/item_sales.csv')


In [3]:
# Identificar todas as colunas que começam com 'd_'
d_columns = [col for col in sales.columns if col.startswith('d_')]

# Exibir as primeiras 10 colunas encontradas para verificação
print(f"Colunas de dias encontradas: {d_columns[:10]}...")

Colunas de dias encontradas: ['d_1', 'd_2', 'd_3', 'd_4', 'd_5', 'd_6', 'd_7', 'd_8', 'd_9', 'd_10']...


In [4]:
def melt_sales(partition):
    """
    Função para derreter a partição do DataFrame sales.
    
    :param partition: Partição do DataFrame Dask.
    :return: DataFrame derretido.
    """
    return partition.melt(
        id_vars=['id', 'item', 'category', 'department', 'store', 'store_code', 'region'],
        value_vars=d_columns,
        var_name='d',
        value_name='sales'
    )


In [5]:
# Aplicar a função melt_sales em cada partição
sales_long = sales.map_partitions(
    melt_sales,
    meta={
        'id': 'object',
        'item': 'object',
        'category': 'object',
        'department': 'object',
        'store': 'object',
        'store_code': 'object',
        'region': 'object',
        'd': 'object',
        'sales': 'int32'
    }
)

In [6]:
# Reparticionar o DataFrame para ter mais partições menores
sales_long = sales_long.repartition(npartitions=100)


In [7]:
# Salvar como múltiplos arquivos Parquet
sales_long.to_parquet('../data/sales_long_parquet/', write_index=False)
